# MNIST

In [0]:
from matrix_square_root_power import *
from shampoo_optimizer import *
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Dropout
from keras.layers import AveragePooling2D, Input, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, Sequential
from keras.datasets import cifar10
from keras import backend
from keras.utils import np_utils
import numpy
from keras.datasets import cifar10
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [0]:
from keras.datasets import mnist
#download mnist data and split into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation="relu"))
model.add(Flatten())
model.add(Dense(10, activation="softmax"))

In [0]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)

In [0]:
def loss(model, x, y, loss_object):
    y_ = model(x)

    return loss_object(y_true=y, y_pred=y_)

def grad(model, inputs, targets, loss_object):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, loss_object)
        return loss_value, tape.gradient(loss_value, model.trainable_weights)

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)
# optimizer = ShampooOptimizer(learning_rate=0.0001)

adam_train_loss_results = []
adam_train_accuracy_results = []

epochs = 20
batch_size = 128

for epoch in range(epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    # Training loop - using batches of 128
    for i in range(X_train.shape[0] // batch_size):
        x = X_train[i*batch_size:i*batch_size+batch_size]
        y = y_train[i*batch_size:i*batch_size+batch_size]
        # Optimize the model
        loss_value, grads = grad(model, x, y, loss_object)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Track progress
        epoch_loss_avg(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        epoch_accuracy(y, model(x))

        # End epoch
    adam_train_loss_results.append(epoch_loss_avg.result())
    adam_train_accuracy_results.append(epoch_accuracy.result())

    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 1.826, Accuracy: 65.042%
Epoch 001: Loss: 1.703, Accuracy: 76.833%
Epoch 002: Loss: 1.692, Accuracy: 77.571%
Epoch 003: Loss: 1.686, Accuracy: 77.995%
Epoch 004: Loss: 1.682, Accuracy: 78.349%
Epoch 005: Loss: 1.679, Accuracy: 78.617%
Epoch 006: Loss: 1.676, Accuracy: 78.829%
Epoch 007: Loss: 1.674, Accuracy: 78.971%
Epoch 008: Loss: 1.672, Accuracy: 79.132%
Epoch 009: Loss: 1.671, Accuracy: 79.235%
Epoch 010: Loss: 1.669, Accuracy: 79.372%
Epoch 011: Loss: 1.662, Accuracy: 80.162%
Epoch 012: Loss: 1.587, Accuracy: 88.014%
Epoch 013: Loss: 1.579, Accuracy: 88.674%
Epoch 014: Loss: 1.576, Accuracy: 88.879%
Epoch 015: Loss: 1.573, Accuracy: 89.071%
Epoch 016: Loss: 1.572, Accuracy: 89.208%
Epoch 017: Loss: 1.570, Accuracy: 89.293%
Epoch 018: Loss: 1.569, Accuracy: 89.378%
Epoch 019: Loss: 1.568, Accuracy: 89.456%


In [0]:
print("MNIST, Adagrad, lr = 10e-4, loss = {0}".format(np.array(adam_train_loss_results)))
print("MNIST, Adagrad, lr = 10e-4, loss = {0}".format(np.array(adam_train_accuracy_results)))

MNIST, Adagrad, lr = 10e-4, loss = [1.8257794 1.7025895 1.6923213 1.686369  1.6823964 1.6788954 1.676232
 1.6741681 1.672348  1.670665  1.6692    1.6620312 1.5872949 1.5789883
 1.5756624 1.5734642 1.5718085 1.5703863 1.5692716 1.5683758]
MNIST, Adagrad, lr = 10e-4, loss = [0.650424   0.7683293  0.7757078  0.77994794 0.7834869  0.78617454
 0.7882946  0.78971356 0.7913161  0.79235107 0.79371995 0.8016159
 0.88014156 0.88673544 0.8887887  0.89070845 0.8920773  0.89292866
 0.89378005 0.8945646 ]
